In [7]:
# 출처1: "https://docs.upbit.com/reference/%EC%A0%84%EC%B2%B4-%EA%B3%84%EC%A2%8C-%EC%A1%B0%ED%9A%8C"
# 출처2: "https://github.com/sharebook-kr/pyupbit"
import requests, time, jwt, uuid, pyupbit, datetime
import pandas as pd
import numpy as np
from sendMail import send_email
from upbit_module import 시장가매수, 지정가매도
import json
from getPriceScaleTick import get_price_scale_tick

f = open("업비트정보.private.json", "r")
api_key = json.load(f)
f.close()

A_key = api_key["accessKey"]  # 본인 access_key 키로 변경
S_key = api_key["secretKey"]  # 본인 secret_key 키로 변경

## 몇 초 간격으로 매도 타이밍을 체크 할 것인지
## 몇 시간 동안 매도 타이밍을 체크 할 것인지
loop_time = 60 * 24 * 100  # 100일 동안 체크

sec = 0 # 시작 값

# 지금 구매한 상태인가? (짧은시간내에 많이 구매되는 현상 방지)
is_already_bought = False

down_bound = 28
up_bound = 65


while True:
    if sec % 3600 == 0:
        send_email("👍", "👍 RSI 1시간 이상무")

    ## 매매할 종목 설정
    krw_tickers = pyupbit.get_tickers(fiat="KRW")
    #market_code_list = ['KRW-BTC', 'KRW-ETH']
    # ==========================================================================================

    for market_code in krw_tickers:
        time.sleep(1)
        print(f'{market_code}를 보고 있습니다.')

        ## currency: 화폐를 의미하는 영문 대문자 코드, balance: 주문가능 금액/수량, locked: 주문 중 묶여있는 금액/수량
        ## avg_buy_price: 매수평균가, avg_buy_price_modified: 매수평균가 수정 여부, unit_currency: 평단가 기준 화폐
        ## API로 업비트에서 내 계좌 조회
        my_exchange_account = pd.DataFrame(requests.get("https://api.upbit.com/v1/accounts",
                                                        headers={"Authorization": 'Bearer {}'.format(
                                                            jwt.encode({'access_key': A_key,
                                                                        'nonce': str(uuid.uuid4())}, S_key))}).json())
        ## 보유 원화
        now_krw = float(my_exchange_account[my_exchange_account['currency'] == 'KRW']['balance'].tail())

        ## API로 업비트에서 15분 단위의 "고가", "시가", "저가", "종가" "거래량" 조회
        ohlcv_value = pyupbit.get_ohlcv(market_code, interval="minute15")
        ## 앞 뒤 "종가" 차이를 변화량 "change"로 지정
        ohlcv_value['change'] = ohlcv_value['close'] - ohlcv_value['close'].shift(1)
        ## 변화량 "change"가 0보다 크거나 같으면 "U"에 "change" 값을 지정하고, 아니면 "U"에 0을 지정
        ohlcv_value['U'] = np.where(ohlcv_value['change'] >= 0, ohlcv_value['change'], 0)
        ## 변화량 "change"가 0보다 작으면 "D"에 "change" 절대값을 지정하고, 아니면 "D"에 0을 지정
        ohlcv_value['D'] = np.where(ohlcv_value['change'] < 0, ohlcv_value['change'].abs(), 0)
        ## "ewm"라는 지수가중함수를 사용하여 평활계수 "alpha"에 "1/14", 계산을 위한 최소 기간 "min_periods"에 "14"를 입력하고
        ## "U"와 "D" 각각의 평균을 구해서 각각 "AU"와 "AD"로 지정
        ohlcv_value['AU'] = ohlcv_value['U'].ewm(alpha=1 / 14, min_periods=14).mean()
        ohlcv_value['AD'] = ohlcv_value['D'].ewm(alpha=1 / 14, min_periods=14).mean()
        ## "AU"를 "AD"로 나눠 "RS"로 지정
        ohlcv_value['RS'] = ohlcv_value['AU'] / ohlcv_value['AD']
        ## "AU"를 "AU"와 "AD"의 합으로 나눠 "RSI"로 지정
        ohlcv_value['RSI'] = ohlcv_value['AU'] / (ohlcv_value['AU'] + ohlcv_value['AD'])
        ## 가장 최근 "RSI"값을 "RSI_value"로 지정
        RSI_value = ohlcv_value[['RSI']].tail(n=1)
        # 0.xx단위로 나와서 100곱해줌
        RSI_value_number = float(RSI_value['RSI']) * 100 
        print(f'15분봉 RSI값은 {RSI_value_number}')
        
        if(RSI_value_number <= down_bound):
            
            my_exchange_account = pd.DataFrame(requests.get("https://api.upbit.com/v1/accounts", headers={"Authorization": 'Bearer {}'.format(jwt.encode({'access_key': A_key,'nonce': str(uuid.uuid4())}, S_key))}).json())
            now_krw = float(my_exchange_account[my_exchange_account['currency'] == 'KRW']['balance'][0])
            if(now_krw >= 5000):
                구매데이터 = 시장가매수(market_code, A_key, S_key)
                내가_구매했던_데이터 = pyupbit.Upbit(A_key, S_key).get_order(구매데이터["uuid"][0])

                # 아직 체결이 안된 경우가 있을 수 있는듯
                while(len(내가_구매했던_데이터["trades"]) == 0):
                    내가_구매했던_데이터 = pyupbit.Upbit(A_key, S_key).get_order(구매데이터["uuid"][0])
                    pass

                내가_구매했던_코인가격 = float(내가_구매했던_데이터["trades"][0]["price"])
                지정가판매가격_temp = 내가_구매했던_코인가격 * 1.02 
                지정가판매가격 = round(지정가판매가격_temp , get_price_scale_tick(지정가판매가격_temp)[0])
                print(f'{내가_구매했던_코인가격}에 구매한거를 {지정가판매가격}에 지정가매도')
                지정가매도(market_code, A_key, S_key, 지정가판매가격)
                time.sleep(60 * 30) # 30분대기
            else:
                print("원화부족")
        
    time.sleep(1)
    sec+=1


KRW-BTC를 보고 있습니다.
15분봉 RSI값은 41.805996110365825
KRW-ETH를 보고 있습니다.
15분봉 RSI값은 51.18763008400534
KRW-NEO를 보고 있습니다.
15분봉 RSI값은 26.44328452048595
KRW-NEO 구매
  currency          balance      locked avg_buy_price  avg_buy_price_modified  \
0      KRW   35798.51498362           0             0                    True   
1     HIVE                0  8.25082508           606                   False   
2   APENFT  512603.10084279           0             0                   False   

  unit_currency  
0           KRW  
1           KRW  
2           KRW  
16560.0에 구매한거를 16890.0에 지정가매도
KRW-NEO 판매
KRW-MTL를 보고 있습니다.
15분봉 RSI값은 55.02468616677159
KRW-XRP를 보고 있습니다.
15분봉 RSI값은 52.88518135771552
KRW-ETC를 보고 있습니다.
15분봉 RSI값은 48.43117103747887
KRW-OMG를 보고 있습니다.
15분봉 RSI값은 48.85968322816626
KRW-SNT를 보고 있습니다.
15분봉 RSI값은 52.18572694551098
KRW-WAVES를 보고 있습니다.
15분봉 RSI값은 51.69061807563151
KRW-XEM를 보고 있습니다.
15분봉 RSI값은 33.93797855188669
KRW-QTUM를 보고 있습니다.
15분봉 RSI값은 38.241442519284206
KRW-LSK를 보고 있습니다.
15분봉 RSI값은 

KRW-MOC를 보고 있습니다.
15분봉 RSI값은 57.75215134224611
KRW-ENJ를 보고 있습니다.
15분봉 RSI값은 45.15669578500156
KRW-TFUEL를 보고 있습니다.
15분봉 RSI값은 47.30453445660077
KRW-MANA를 보고 있습니다.
15분봉 RSI값은 52.58364344371289
KRW-ANKR를 보고 있습니다.
15분봉 RSI값은 37.19812431047447
KRW-AERGO를 보고 있습니다.
15분봉 RSI값은 48.99094614989119
KRW-ATOM를 보고 있습니다.
15분봉 RSI값은 55.916962829258345
KRW-TT를 보고 있습니다.
15분봉 RSI값은 53.60742274916219
KRW-CRE를 보고 있습니다.
15분봉 RSI값은 45.90111307009815
KRW-MBL를 보고 있습니다.
15분봉 RSI값은 59.359202402528545
KRW-WAXP를 보고 있습니다.
15분봉 RSI값은 55.07088201947394
KRW-HBAR를 보고 있습니다.
15분봉 RSI값은 47.60746582743799
KRW-MED를 보고 있습니다.
15분봉 RSI값은 52.27616435622154
KRW-MLK를 보고 있습니다.
15분봉 RSI값은 59.74655078870995
KRW-STPT를 보고 있습니다.
15분봉 RSI값은 51.71785555222036
KRW-ORBS를 보고 있습니다.
15분봉 RSI값은 51.973204649701444
KRW-VET를 보고 있습니다.
15분봉 RSI값은 44.21399666909258
KRW-CHZ를 보고 있습니다.
15분봉 RSI값은 64.14201962253479
KRW-STMX를 보고 있습니다.
15분봉 RSI값은 54.679811513839894
KRW-DKA를 보고 있습니다.
15분봉 RSI값은 49.136814660893236
KRW-HIVE를 보고 있습니다.
15분봉 RSI값은 55.26104884018

KRW-NEAR를 보고 있습니다.
15분봉 RSI값은 45.56686724631114
KRW-AVAX를 보고 있습니다.
15분봉 RSI값은 50.231519486535994
KRW-T를 보고 있습니다.
15분봉 RSI값은 45.5748325758234
KRW-CELO를 보고 있습니다.
15분봉 RSI값은 54.287134331469446
KRW-GMT를 보고 있습니다.
15분봉 RSI값은 41.89249221625968
KRW-APT를 보고 있습니다.
15분봉 RSI값은 50.944823682435846
KRW-SHIB를 보고 있습니다.
15분봉 RSI값은 51.630410724632995
KRW-BTC를 보고 있습니다.
15분봉 RSI값은 46.65371607649828
KRW-ETH를 보고 있습니다.
15분봉 RSI값은 58.23288883278267
KRW-NEO를 보고 있습니다.
15분봉 RSI값은 35.35224691786002
KRW-MTL를 보고 있습니다.
15분봉 RSI값은 55.02468135704449
KRW-XRP를 보고 있습니다.
15분봉 RSI값은 47.083707852360156
KRW-ETC를 보고 있습니다.
15분봉 RSI값은 47.51671087641196
KRW-OMG를 보고 있습니다.
15분봉 RSI값은 41.8858871910717
KRW-SNT를 보고 있습니다.
15분봉 RSI값은 52.1857255020039
KRW-WAVES를 보고 있습니다.
15분봉 RSI값은 55.07401354359196
KRW-XEM를 보고 있습니다.
15분봉 RSI값은 34.60737206440111
KRW-QTUM를 보고 있습니다.
15분봉 RSI값은 42.45498593939604
KRW-LSK를 보고 있습니다.
15분봉 RSI값은 46.88769468911489
KRW-STEEM를 보고 있습니다.
15분봉 RSI값은 56.632753564787365
KRW-XLM를 보고 있습니다.
15분봉 RSI값은 41.756518848697084
KR

KRW-MANA를 보고 있습니다.
15분봉 RSI값은 48.33691949387966
KRW-ANKR를 보고 있습니다.
15분봉 RSI값은 36.41243613498871
KRW-AERGO를 보고 있습니다.
15분봉 RSI값은 53.33799383174941
KRW-ATOM를 보고 있습니다.
15분봉 RSI값은 46.13955833852471
KRW-TT를 보고 있습니다.
15분봉 RSI값은 53.60742399721773
KRW-CRE를 보고 있습니다.
15분봉 RSI값은 43.27468927691858
KRW-MBL를 보고 있습니다.
15분봉 RSI값은 59.35920103282102
KRW-WAXP를 보고 있습니다.
15분봉 RSI값은 50.64279756450567
KRW-HBAR를 보고 있습니다.
15분봉 RSI값은 42.16668898675493
KRW-MED를 보고 있습니다.
15분봉 RSI값은 52.27616198282566
KRW-MLK를 보고 있습니다.
15분봉 RSI값은 50.91305540461239
KRW-STPT를 보고 있습니다.
15분봉 RSI값은 47.992023562497074
KRW-ORBS를 보고 있습니다.
15분봉 RSI값은 48.373945091701884
KRW-VET를 보고 있습니다.
15분봉 RSI값은 47.27442367195921
KRW-CHZ를 보고 있습니다.
15분봉 RSI값은 61.00714310338342
KRW-STMX를 보고 있습니다.
15분봉 RSI값은 51.237558921177104
KRW-DKA를 보고 있습니다.
15분봉 RSI값은 49.13681336124363
KRW-HIVE를 보고 있습니다.
15분봉 RSI값은 51.69271001652703
KRW-KAVA를 보고 있습니다.
15분봉 RSI값은 48.857515734140264
KRW-AHT를 보고 있습니다.
15분봉 RSI값은 57.37568448746242
KRW-LINK를 보고 있습니다.
15분봉 RSI값은 44.804336154178

KRW-CELO를 보고 있습니다.
15분봉 RSI값은 42.65796108447757
KRW-GMT를 보고 있습니다.
15분봉 RSI값은 41.892487893250085
KRW-APT를 보고 있습니다.
15분봉 RSI값은 44.800681225863535
KRW-SHIB를 보고 있습니다.
15분봉 RSI값은 44.91124591413258
KRW-BTC를 보고 있습니다.
15분봉 RSI값은 39.58081227942505
KRW-ETH를 보고 있습니다.
15분봉 RSI값은 45.314134446743104
KRW-NEO를 보고 있습니다.
15분봉 RSI값은 31.093333920185213
KRW-MTL를 보고 있습니다.
15분봉 RSI값은 50.456974973915955
KRW-XRP를 보고 있습니다.
15분봉 RSI값은 47.083707852360156
KRW-ETC를 보고 있습니다.
15분봉 RSI값은 37.163429737613576
KRW-OMG를 보고 있습니다.
15분봉 RSI값은 38.896488358814246
KRW-SNT를 보고 있습니다.
15분봉 RSI값은 52.1857255020039
KRW-WAVES를 보고 있습니다.
15분봉 RSI값은 48.070403349509235
KRW-XEM를 보고 있습니다.
15분봉 RSI값은 33.31531194060165
KRW-QTUM를 보고 있습니다.
15분봉 RSI값은 36.67955937947614
KRW-LSK를 보고 있습니다.
15분봉 RSI값은 41.82376388743943
KRW-STEEM를 보고 있습니다.
15분봉 RSI값은 41.600975187983295
KRW-XLM를 보고 있습니다.
15분봉 RSI값은 41.756518848697084
KRW-ARDR를 보고 있습니다.
15분봉 RSI값은 46.33957111325196
KRW-ARK를 보고 있습니다.
15분봉 RSI값은 58.4463845566041
KRW-STORJ를 보고 있습니다.
15분봉 RSI값은 42.507904335

...

KRW-ATOM를 보고 있습니다.
15분봉 RSI값은 49.484873959529054
KRW-TT를 보고 있습니다.
15분봉 RSI값은 52.97995395731604
KRW-CRE를 보고 있습니다.
15분봉 RSI값은 56.74862993882839
KRW-MBL를 보고 있습니다.
15분봉 RSI값은 54.5178242878701
KRW-WAXP를 보고 있습니다.
15분봉 RSI값은 48.119587853317114
KRW-HBAR를 보고 있습니다.
15분봉 RSI값은 52.61172155671304
KRW-MED를 보고 있습니다.
15분봉 RSI값은 50.917124160365944
KRW-MLK를 보고 있습니다.
15분봉 RSI값은 66.32991035071409
KRW-STPT를 보고 있습니다.
15분봉 RSI값은 61.78289977358874
KRW-ORBS를 보고 있습니다.
15분봉 RSI값은 56.87211762188791
KRW-VET를 보고 있습니다.
15분봉 RSI값은 46.33383336210783
KRW-CHZ를 보고 있습니다.
15분봉 RSI값은 56.09246699614437
KRW-STMX를 보고 있습니다.
15분봉 RSI값은 57.80365101720215
KRW-DKA를 보고 있습니다.
15분봉 RSI값은 62.328330623459024
KRW-HIVE를 보고 있습니다.
15분봉 RSI값은 56.47222326632999
KRW-KAVA를 보고 있습니다.
15분봉 RSI값은 52.46011027451257
KRW-AHT를 보고 있습니다.
15분봉 RSI값은 45.52838123731342
KRW-LINK를 보고 있습니다.
15분봉 RSI값은 60.74931756437954
KRW-XTZ를 보고 있습니다.
15분봉 RSI값은 55.502647656913105
KRW-BORA를 보고 있습니다.
15분봉 RSI값은 52.453601743104585
KRW-JST를 보고 있습니다.
15분봉 RSI값은 53.93969407858947

KRW-SHIB를 보고 있습니다.
15분봉 RSI값은 52.19001384881467
KRW-BTC를 보고 있습니다.
15분봉 RSI값은 55.8291497309859
KRW-ETH를 보고 있습니다.
15분봉 RSI값은 60.362179241974665
KRW-NEO를 보고 있습니다.
15분봉 RSI값은 57.121577216864225
KRW-MTL를 보고 있습니다.
15분봉 RSI값은 53.46050069559429
KRW-XRP를 보고 있습니다.
15분봉 RSI값은 55.22134759583515
KRW-ETC를 보고 있습니다.
15분봉 RSI값은 58.447263641573585
KRW-OMG를 보고 있습니다.
15분봉 RSI값은 54.50997381140126
KRW-SNT를 보고 있습니다.
15분봉 RSI값은 57.36689881154026
KRW-WAVES를 보고 있습니다.
15분봉 RSI값은 55.761682001475286
KRW-XEM를 보고 있습니다.
15분봉 RSI값은 43.55441703226751
KRW-QTUM를 보고 있습니다.
15분봉 RSI값은 57.51467159523044
KRW-LSK를 보고 있습니다.
15분봉 RSI값은 49.19212867141979
KRW-STEEM를 보고 있습니다.
15분봉 RSI값은 60.25762004097933
KRW-XLM를 보고 있습니다.
15분봉 RSI값은 57.3221705897616
KRW-ARDR를 보고 있습니다.
15분봉 RSI값은 63.20990519275915
KRW-ARK를 보고 있습니다.
15분봉 RSI값은 61.89078535812844
KRW-STORJ를 보고 있습니다.
15분봉 RSI값은 54.547406914223
KRW-GRS를 보고 있습니다.
15분봉 RSI값은 54.61146793438978
KRW-REP를 보고 있습니다.
15분봉 RSI값은 66.79356283151299
KRW-ADA를 보고 있습니다.
15분봉 RSI값은 50.1233495884743
KRW-S

KRW-MBL를 보고 있습니다.
15분봉 RSI값은 58.73697102019505
KRW-WAXP를 보고 있습니다.
15분봉 RSI값은 55.94436065614767
KRW-HBAR를 보고 있습니다.
15분봉 RSI값은 57.309340219239914
KRW-MED를 보고 있습니다.
15분봉 RSI값은 50.917124160365944
KRW-MLK를 보고 있습니다.
15분봉 RSI값은 69.77452902142326
KRW-STPT를 보고 있습니다.
15분봉 RSI값은 68.43603835229098
KRW-ORBS를 보고 있습니다.
15분봉 RSI값은 61.09907515576175
KRW-VET를 보고 있습니다.
15분봉 RSI값은 53.57366172946267
KRW-CHZ를 보고 있습니다.
15분봉 RSI값은 56.09246699614437
KRW-STMX를 보고 있습니다.
15분봉 RSI값은 57.80365101720215
KRW-DKA를 보고 있습니다.
15분봉 RSI값은 62.328330623459024
KRW-HIVE를 보고 있습니다.
15분봉 RSI값은 59.464644318567395
KRW-KAVA를 보고 있습니다.
15분봉 RSI값은 56.93555313437013
KRW-AHT를 보고 있습니다.
15분봉 RSI값은 45.52838123731342
KRW-LINK를 보고 있습니다.
15분봉 RSI값은 64.40570347104952
KRW-XTZ를 보고 있습니다.
15분봉 RSI값은 60.0217539136626
KRW-BORA를 보고 있습니다.
15분봉 RSI값은 47.253955726902696
KRW-JST를 보고 있습니다.
15분봉 RSI값은 59.68086631554059
KRW-CRO를 보고 있습니다.
15분봉 RSI값은 61.493522641041565
KRW-TON를 보고 있습니다.
15분봉 RSI값은 76.99841555612915
KRW-SXP를 보고 있습니다.
15분봉 RSI값은 54.14597346019965

KRW-NEO를 보고 있습니다.
15분봉 RSI값은 58.01793194862091
KRW-MTL를 보고 있습니다.
15분봉 RSI값은 56.97820516344674
KRW-XRP를 보고 있습니다.
15분봉 RSI값은 55.22134759583515
KRW-ETC를 보고 있습니다.
15분봉 RSI값은 53.71724310905526
KRW-OMG를 보고 있습니다.
15분봉 RSI값은 54.50997381140126
KRW-SNT를 보고 있습니다.
15분봉 RSI값은 53.685917022847896
KRW-WAVES를 보고 있습니다.
15분봉 RSI값은 55.761682001475286
KRW-XEM를 보고 있습니다.
15분봉 RSI값은 43.55441703226751
KRW-QTUM를 보고 있습니다.
15분봉 RSI값은 53.70264293055206
KRW-LSK를 보고 있습니다.
15분봉 RSI값은 58.81084080251914
KRW-STEEM를 보고 있습니다.
15분봉 RSI값은 60.25762004097933
KRW-XLM를 보고 있습니다.
15분봉 RSI값은 57.3221705897616
KRW-ARDR를 보고 있습니다.
15분봉 RSI값은 63.20990519275915
KRW-ARK를 보고 있습니다.
15분봉 RSI값은 57.478046360192906
KRW-STORJ를 보고 있습니다.
15분봉 RSI값은 54.547406914223
KRW-GRS를 보고 있습니다.
15분봉 RSI값은 50.46204346573509
KRW-REP를 보고 있습니다.
15분봉 RSI값은 65.49948305033982
KRW-ADA를 보고 있습니다.
15분봉 RSI값은 50.1233495884743
KRW-SBD를 보고 있습니다.
15분봉 RSI값은 64.00763270643135
KRW-POWR를 보고 있습니다.
15분봉 RSI값은 55.85571826898856
KRW-BTG를 보고 있습니다.
15분봉 RSI값은 56.54815979699477
KRW-I

KeyboardInterrupt: 